### ספריות


In [1]:
import pandas as pd
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### העלת משתנים להרצת הקוד


In [ ]:
# מיקום תיקייה נוכחית
cwd = os.getcwd()

In [ ]:
# תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

### פונקציות גלובליות


In [3]:
from global_functions import remove_spaces_in_columns, up_load_df, split_to_layers

### העלאת טבלאות


In [ ]:
# בתי ספר וגנים מעיריית בית שמש
BShemesh=up_load_df(r'{}\background_files\betshemesh_muni\חינוך'.format(cwd),'מצבת מוסדות חינוך תשפג משרדי')
BShemesh=remove_spaces_in_columns(BShemesh)

### עיבוד


In [5]:
# מחיקת שורות של גני ילדים
BShemesh=BShemesh[~BShemesh['הערות'].str.contains('גן', na=False)]

In [ ]:
# קאורדינטות של מוסדות חינוך במרחב ירושלים
JTMT_setls_schools_coordinates_with_src=up_load_df(r'{}\background_files'.format(cwd),'JTMT_setls_schools_coordinates_with_src')
JTMT_setls_schools_coordinates_with_src=remove_spaces_in_columns(JTMT_setls_schools_coordinates_with_src)

In [7]:
# בדיקה אם יש מוסדות עם אותו סמל מוסד
is_unique = JTMT_setls_schools_coordinates_with_src['סמל_מוסד'].is_unique

In [8]:
# הוספת עמודת SRC
BShemesh['SRC'] = ''

In [ ]:
# מיזוג טבלאות המוסדות והקאורדינטות לפי העמודה "סמל_מוסד"
BShemesh = pd.merge(BShemesh, JTMT_setls_schools_coordinates_with_src[['סמל_מוסד', 'coordinate_x', 'coordinate_y', 'SRC']],
                     on='סמל_מוסד', how='left')

In [ ]:
# מחיקת עמודות מיותרות
BShemesh=BShemesh.drop(columns=['שכונה', 'כתובת', 'טלפון', 'פקס', 'מייל', 'שם_מנהל/ת',
                                    'כתובת_מנהל', 'נייד', 'שם_מזכיר/ה', 'נייד.1', 'הערות', 'מפקח', 'קב"ט_מוס"ח',
                                    'גן_טרום_חובה', 'גן_חובה', 'גני_חינוך_מיוחד', 'סה"כ_גנים'])

In [ ]:
# רשימת שמות השכבות (א' עד יב')
grades = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב']

# לולאה שמבצעת את הסכימה של מספר התלמידים עבור כל שכבה
for i, grade in enumerate(grades):
    # איתור העמודות הרלוונטיות לכל שכבה (כולל כיתה מקדמת אם קיימת לאותה שכבה)
    grade_columns = [col for col in BShemesh.columns if col.startswith(f'כיתה_{grade}')]
    
    # הוספת העמודה של הכיתה המקדמת של אותה שכבה אם קיימת
    if f'כיתה_מקדמת.{i}' in BShemesh.columns:
        grade_columns.append(f'כיתה_מקדמת.{i}')
    elif 'כיתה_מקדמת' in BShemesh.columns and i == 0:
        grade_columns.append('כיתה_מקדמת')
    
    # סכימת העמודות אם יש כיתות רלוונטיות לשכבה הזו
    if grade_columns:
        BShemesh[f'כיתה_{grade}'] = BShemesh[grade_columns].sum(axis=1, skipna=True)
        
        # מחיקת העמודות שסוכמו
        BShemesh.drop(columns=grade_columns, inplace=True)

In [12]:
# יצירת עמודה "סהכ_תלמידים" שמכילה את סך התלמידים במוסד
BShemesh['סה"כ_תלמידים'] = BShemesh[['סה"כ_יסודי_מ"מ_וממ"ד', 'סה"כ_יסודי_חרדי', '_סה"כ_על_יסודי_מ"מ_וממ"ד', 'סה"כ_על_יסודי_חרדי']].sum(axis=1)

# הסרת העמודות הישנות
BShemesh=BShemesh.drop(columns=['סה"כ_יסודי_מ"מ_וממ"ד', 'סה"כ_יסודי_חרדי', '_סה"כ_על_יסודי_מ"מ_וממ"ד', 'סה"כ_על_יסודי_חרדי'])

In [13]:
# חלוקה למגזרים
BShemesh.loc[BShemesh['מגזר']=='חרדי','sector'] = 2
BShemesh.loc[BShemesh['מגזר']=='ממ"ח','sector'] = 2
BShemesh.loc[BShemesh['מגזר']=='מ"מ','sector'] = 3
BShemesh.loc[BShemesh['מגזר']=='ממ"ד','sector'] = 3
BShemesh.loc[BShemesh['מגזר']=='עירוני','sector'] = 3

In [14]:
# מחיקת שורות שבהן העמודה "סהכ_תלמידים" שווה לאפס
BShemesh=BShemesh[BShemesh['סה"כ_תלמידים'] != 0]

In [15]:
# חלוקה לשכבות גיל
ages = {
    'ele': ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו'],
    'mid': ['כיתה_ז', 'כיתה_ח', 'כיתה_ט'],
    'high': ['כיתה_י', 'כיתה_יא', 'כיתה_יב']
}

# חישוב מספר התלמידים לכל שכבה
BShemesh = split_to_layers(BShemesh, ages)

BShemesh = BShemesh.drop(['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב'], axis=1)

In [16]:
# בדיקה אם יש מוסדות עם אותו סמל מוסד
is_unique = BShemesh['סמל_מוסד'].is_unique

In [17]:
# מציאת שורות עם ערכים כפולים בעמודה סמל_מוסד
duplicated_rows = BShemesh[BShemesh.duplicated(subset=['סמל_מוסד'], keep=False)]

In [ ]:
# קיבוץ שורות עם ערכים כפולים בעמודה סמל_מוסד
duplicated_rows = duplicated_rows.groupby('סמל_מוסד').agg({
    'מגזר': 'first',  # ערך ראשון
    'שם_המוסד': 'first',  # ערך ראשון
    'coordinate_x': 'first',  # ערך ראשון
    'coordinate_y': 'first',  # ערך ראשון
    'SRC_y': 'first',  # ערך ראשון
    'סה"כ_תלמידים': 'sum',  # ערך ראשון
    'sector': 'first',  # ערך ראשון
    'ele_stu': 'sum',    # סכימה
    'mid_stu': 'sum',    # סכימה
    'high_stu': 'sum',    # סכימה
}).reset_index()

In [ ]:
# חיבור של הטבלה המקורית עם הטבלה של המוסדות עם אותו סמל מוסד אחרי קיבוץ
BShemesh = pd.concat([duplicated_rows, BShemesh]).drop_duplicates(subset=['סמל_מוסד'], keep='first').reset_index(drop=True)

In [ ]:
# שינוי שם של העמודות
BShemesh = BShemesh.rename(columns={
    'סמל_מוסד': 'ID',
    'coordinate_x': 'x',
    'coordinate_y': 'y',
    'SRC_y': 'SRC'
})

# מחיקת עמודות מיותרות
BShemesh = BShemesh.drop(['מגזר', 'סה"כ_תלמידים', 'שם_המוסד', 'SRC_x'], axis=1)

### ייצוא


In [ ]:
BShemesh.to_excel(r'{}\Intermediates\{}_BShemesh_final.xlsx'.format(cwd, file_date), index=False)